# Feature Selection Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install xgboost --upgrade --quiet

In [ ]:
!pip install mpl-chord-diagram
!pip install eli5

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import resample
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split, learning_curve, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

from scipy.stats import spearmanr, pearsonr, false_discovery_control, linregress
from xgboost import XGBRegressor

import re

In [ ]:
def get_simple_feature_name(feature_names):
    simple_names = []
    for feature_name in feature_names:
        simple_name=feature_name
        pattern = r'\((.*)'
        match = re.search(pattern, feature_name)
        if match:
            simple_name = match.group(1)
            if simple_name in simple_names:
                last_underscore_index = feature_name.rfind('_')
                simple_name = feature_name[last_underscore_index:] if last_underscore_index != -1 else feature_name
        simple_names.append(simple_name)
    return simple_names

In [ ]:
def one_hot_encoding(df, feature_name):
    if feature_name in df.columns:
        encoder = OneHotEncoder(sparse_output=False)
        encoded_data = encoder.fit_transform(df[[feature_name]])
        one_hot_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([feature_name]))
        df = pd.concat([df, one_hot_df], axis=1)
        df = df.drop(feature_name, axis=1)
    return df

In [ ]:
def hist_plot(df):
    df_renamed = df.copy()
    df_renamed.columns = get_simple_feature_name(df.columns)
    df_renamed.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)
    plt.grid(True, linewidth=0.5)
    plt.show()

In [ ]:
def pair_plot(X, y):
    X_renamed = X.copy()
    X_renamed.columns = get_simple_feature_name(X.columns)
    y_df = y.to_frame().rename(columns={0: 'Dose'})
    df = pd.concat([X_renamed, y_df], axis=1)

    num_features = len(X.columns)
    num_cols = 3
    num_rows = (num_features + num_cols - 1) // num_cols
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 5 * num_rows))

    for i, ax in enumerate(axes.flat):
        if i < num_features:
            feature = df.columns[i]
            ax.scatter(df[feature], df['DOSE'])
            slope, intercept, r_value, p_value, std_err = linregress(df[feature], df['DOSE'])
            ax.plot(df[feature], slope * df[feature] + intercept, color='green')
            ax.set_xlabel(feature)
            ax.set_ylabel('Dose')
            ax.text(0.05, 0.95, f'r = {r_value:.3f}\np = {p_value:.3f}', transform=ax.transAxes, va='top')
        else:
            ax.axis('off')
    plt.grid(True, linewidth=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_chord_diagram(X):
    fig, ax = plt.subplots(figsize=(12, 12))
    chord_diagram(X.corr(), names=get_simple_feature_name(X.columns), ax=ax, rotation=0, fontsize=11)
    plt.show()

In [ ]:
def plot_learning_curve(X, y, model, cv):
    fig, ax = plt.subplots(figsize=(10, 6), sharex=True)

    common_params = {
        "X":X,
        "y": y,
        "train_sizes": np.linspace(0.1, 1.0, 5, 20),
        "cv": cv,
        "n_jobs": 4,
        "return_times": True,
        "scoring":'neg_mean_squared_error'
    }
    train_sizes, train_scores, test_scores, fit_times, score_times = learning_curve(
        model, **common_params
    )

    ax.plot(fit_times.mean(axis=1), -train_scores.mean(axis=1),"o-", label="Training Score")
    ax.plot(fit_times.mean(axis=1), -test_scores.mean(axis=1), "o-", label="Validation Score")
    ax.fill_between(
        fit_times.mean(axis=1),
        train_scores.mean(axis=1) - train_scores.std(axis=1),
        train_scores.mean(axis=1) + train_scores.std(axis=1),
        alpha=0.3,
    )
    ax.fill_between(
        fit_times.mean(axis=1),
        test_scores.mean(axis=1) - test_scores.std(axis=1),
        test_scores.mean(axis=1) + test_scores.std(axis=1),
        alpha=0.3,
    )
    ax.set_ylabel("MSE score")
    ax.set_xlabel("Fit time (s)")
    ax.set_title(
        f"Performance of the {model.__class__.__name__} classifier"
    )
    plt.grid(True, linewidth=0.5)
    plt.legend()
    plt.show()

In [ ]:
def pca_analysis(X_selected):

    pca = PCA()
    X_pca = pca.fit_transform(StandardScaler().fit_transform(X_selected))

    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
    plt.xlabel('Number of principal components')
    plt.ylabel('Explained variance ratio')
    plt.title('Scree plot')
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], s=20, alpha=0.5)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('Scatter plot of the first two principal components')
    plt.show()

    plt.tight_layout()

    plt.show()

In [ ]:
def plot_prediction_graph(y, y_pred, r2):
    plt.figure(figsize=(10, 6))
    plt.scatter(y, y_pred, color='#1f77b4', alpha=0.6)
    plt.plot(np.linspace(min(y), max(y), 100), np.linspace(min(y), max(y), 100), color='red', linewidth=2)
    plt.xlabel('Measured Dose(GY/GBq)', fontsize=14)
    plt.ylabel('Predicted Dose(GY/GBq)', fontsize=14)
    plt.title(f"Predicted vs. Actual (R-squared: {r2:.3f})", fontsize=16, color='#333333')
    plt.text(0.05, 0.95, f"R²: {r2:.3f}", transform=plt.gca().transAxes, va='top', fontsize=11, color='#333333')
    plt.grid(True, linewidth=0.5)
    plt.show()

In [ ]:
def plot_prediction_graph2(y, y_pred, r2):
    plt.figure(figsize=(10, 6))
    sns.regplot(x=y, y=y_pred,
            scatter_kws=dict(color='#00008B', alpha=0.6),
            line_kws=dict(color='#FE420F'))
    plt.plot(np.linspace(min(y), max(y), 100), np.linspace(min(y), max(y), 100), color='black', linestyle='--', label='Identity Line')
    plt.xlabel('Measured Dose (Gy/ GBq)', fontsize=12)
    plt.ylabel('Predicted Dose (Gy/ GBq)', fontsize=12)
    plt.title(f'R²: {r2:.2f}', fontsize=12)
    plt.legend(loc='upper left', fontsize=10)
    plt.grid(True, linewidth=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
def calculate_scores(y, y_pred, r2_required=True):
    r2 = -1
    if (r2_required):
        r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)
    return r2, mae, mse, mape

In [ ]:
def print_scores(mode, r2_scores, mae_scores, mse_scores, rmse_scores, mape_scores, corr_score):
    print("\nMean", mode, "Scores:")
    print("Mean R2 Score:", np.mean(r2_scores))
    print("Mean MAE:", np.mean(mae_scores))
    print("Mean MSE:", np.mean(mse_scores))
    print("Mean RMSE:", np.mean(rmse_scores))
    print("Mean MAPE:", np.mean(mape_scores))
    print("Spearman Correlation:", corr_score)
    print()

In [ ]:
def p_value_correction(X, y, alpha=0.05):
    p_values = []
    for col in X.columns:
        _, p = spearmanr(X[col], y)
        p_values.append(p)

    corrected_p_values = false_discovery_control(p_values, method='bh')

    print("Corrected p-values:")
    for feature, p_value in zip(X.columns, corrected_p_values):
        print(f"{feature}: {p_value:.10f}")

    selected_features = [feature for feature, p_val in zip(X.columns, corrected_p_values) if p_val < alpha]
    removed_features = [feature for feature, p_val in zip(X.columns, corrected_p_values) if p_val >= alpha]

    print()
    print("Removed features:")
    print(removed_features)
    return X[selected_features]

In [ ]:
def evaluate_model(X_selected, y, model, cv, print_results=True):

    r2_train_scores = []
    mae_train_scores = []
    mse_train_scores = []
    rmse_train_scores = []
    mape_train_scores = []

    r2_test_scores = []
    mae_test_scores = []
    mse_test_scores = []
    rmse_test_scores = []
    mape_test_scores = []

    y_tests=[]
    y_test_preds=[]

    y_trains=[]
    y_train_preds=[]

    for train_idx, test_idx in cv.split(X_selected, y):

        X_train, X_test = X_selected[train_idx], X_selected[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)


        y_tests += list(y_test)
        y_test_preds += list(y_test_pred)

        y_trains += list(y_train)
        y_train_preds += list(y_train_pred)

        r2_train, mae_train, mse_train, mape_train = calculate_scores(y_train, y_train_pred)

        if isinstance(cv, LeaveOneOut):
            r2_test, mae_test, mse_test, mape_test = calculate_scores(y_test, y_test_pred, False)
        else:
            r2_test, mae_test, mse_test, mape_test = calculate_scores(y_test, y_test_pred)

        r2_train_scores.append(r2_train)
        mae_train_scores.append(mae_train)
        mse_train_scores.append(mse_train)
        rmse_train_scores.append(np.sqrt(mse_train))
        mape_train_scores.append(mape_train)

        r2_test_scores.append(r2_test)
        mae_test_scores.append(mae_test)
        mse_test_scores.append(mse_test)
        rmse_test_scores.append(np.sqrt(mse_test))
        mape_test_scores.append(mape_test)

    corr_train, _ = spearmanr(y_trains, y_train_preds)
    corr_test, _ = spearmanr(y_tests, y_test_preds)

    if isinstance(cv, LeaveOneOut):
        r2_test_scores = r2_score(y_tests, y_test_preds)

    if (print_results):
        print_scores('Train', r2_train_scores, mae_train_scores, mse_train_scores, rmse_train_scores, mape_train_scores, corr_train)

        print_scores('Test', r2_test_scores, mae_test_scores, mse_test_scores, rmse_test_scores, mape_test_scores, corr_test)

    return np.mean(r2_test_scores), np.mean(r2_train_scores), np.mean(mse_test_scores), y_trains, y_train_preds, y_tests, y_test_preds

In [ ]:

def evaluate_based_on_feature_number2(X_sorted, y, model, cv):
    num_features_list = []
    r2_scores = []
    mse_scores = []

    for num_features in range(X_sorted.shape[1], 1, -1):
        X_selected = X_sorted[:, :num_features]

        r2, mse, y_tests, y_pred_tests = evaluate_model(X_selected, y, model, cv, print_results=False)

        num_features_list.append(num_features)
        r2_scores.append(r2)
        mse_scores.append(mse)

    plt.figure(figsize=(10, 6))
    plt.plot(num_features_list, r2_scores, marker='o', label='R^2 Score')
    plt.xlabel('Number of Selected Features')
    plt.ylabel('R^2 Score')
    plt.title('R^2 Score vs. Number of Selected Features')
    plt.xticks(num_features_list)
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(num_features_list, mse_scores, marker='x', label='Mean Squared Error')
    plt.xlabel('Number of Selected Features')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Number of Selected Features')
    plt.xticks(num_features_list)
    plt.legend()
    plt.grid(True, linewidth=0.5)
    plt.show()

In [ ]:

def evaluate_based_on_feature_number2(X_sorted, y, model, cv):
    num_features_list = []
    r2_scores = []
    mse_scores = []

    for num_features in range(X_sorted.shape[1], 1, -1):
        X_selected = X_sorted[:, :num_features]

        r2, mse, y_tests, y_pred_tests = evaluate_model(X_selected, y, model, cv, print_results=False)

        num_features_list.append(num_features)
        r2_scores.append(r2)
        mse_scores.append(mse)

    plt.figure(figsize=(10, 6))
    plt.plot(num_features_list, r2_scores, marker='o', label='R^2 Score')
    plt.xlabel('Number of Selected Features')
    plt.ylabel('R^2 Score')
    plt.title('R^2 Score vs. Number of Selected Features')
    plt.xticks(num_features_list)
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(num_features_list, mse_scores, marker='x', label='Mean Squared Error')
    plt.xlabel('Number of Selected Features')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Error vs. Number of Selected Features')
    plt.xticks(num_features_list)
    plt.legend()
    plt.grid(True, linewidth=0.5)
    plt.show()

In [ ]:
def fit(self, X, y):
    self.model_ = self.build_fn()
    y = np.array(y).reshape(-1, 1)
    self.model_.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
    return self

def predict(self, X):
    predictions = self.model_.predict(X)
    return predictions.flatten()

In [ ]:
df1 = pd.read_excel("/content/drive/MyDrive/dosiomics/LDVK.xlsx")
df2 = pd.read_excel("/content/drive/MyDrive/dosiomics/LCT.xlsx")

df1.columns = [col + '_DRM' for col in df1.columns]

df2.drop(['DOSE'], axis=1, inplace=True)
df2.columns = [col + '_CT' for col in df2.columns]
#df = pd.concat([df1, df2], axis=1)

# clinical.drop(columns=['PATIENT', 'DOSE'], axis=1, inplace=True)

# pet_ct_clinical = pd.concat([pet_ct, clinical_imputed], axis=1)

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

df0 = pd.read_excel("/content/drive/MyDrive/dosiomics/LB.xlsx")

numeric_cols = df0.select_dtypes(include='number').columns.tolist()

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numeric_cols),
    ])

df3 = pd.DataFrame(preprocessor.fit_transform(df0), columns=numeric_cols)
df3.drop(['DOSE'], axis=1, inplace=True)
df3.columns = [col + '_CB' for col in df3.columns]

df = pd.concat([df1,df2,df3], axis=1)
df4= pd.concat([df1,df2], axis=1)
#df1: DVK only
#df2: CT only
#df3: CB only
#df4: DVK+CT
#df: all


#E1: DVK + CT ....>feature selection: df4
#E2: DVK+CT+CB....>feature selection: df
#E3: result E1 + whole CBs: selected features df4 + d0 (whole CBs)

In [ ]:
y=df['DOSE_DRM']
X=df.drop('DOSE_DRM', axis=1)

In [ ]:
X_scaled = StandardScaler().fit_transform(X)
X_selected = X
X_selected_encoded = one_hot_encoding(X_selected, 'LOC')
X_selected_scaled = StandardScaler().fit_transform(X_selected_encoded)

**RFE feature selection**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor

from sklearn.feature_selection import RFE

estimator = GradientBoostingRegressor()
num_features = 10

rfe = RFE(estimator, n_features_to_select=num_features)
rfe.fit(X_selected_scaled, y)

selected_features = X.columns[rfe.support_]

for f, feature in enumerate(selected_features):
    print("%d. Feature %s" % (f + 1, feature))

estimator.fit(X_selected_scaled, y)

feature_importances = estimator.feature_importances_

selected_feature_importance = [(selected_features[i], feature_importances[X.columns.get_loc(selected_features[i])])
                               for i in range(num_features)]

selected_feature_importance.sort(key=lambda x: x[1], reverse=True)

rfe_final_features, importances = zip(*selected_feature_importance)

plt.figure(figsize=(10, 6))
plt.barh(rfe_final_features, importances, color='#993366')
plt.xlabel('Feature Importance')
plt.title('Feature Importance of Selected Features after RFE')
plt.gca().invert_yaxis()
plt.show()

Creating X_selected_sorted_df

In [ ]:
encoded_cols = X_selected_encoded.columns
name_to_idx = {col: i for i, col in enumerate(encoded_cols)}

idx_final = np.array([name_to_idx[f] for f in rfe_final_features], dtype=int)

rfe_ranks_kept = rfe.ranking_[rfe.support_]
order = np.argsort(rfe_ranks_kept)
idx_final_sorted = idx_final[order]

X_selected_sorted = X_selected_scaled[:, idx_final_sorted]

print("Column indices of RFE final features:", idx_final_sorted)
print("Ordered feature names:", encoded_cols[idx_final_sorted].tolist())

X_selected_sorted_df = pd.DataFrame(
    X_selected_sorted,
    columns=encoded_cols[idx_final_sorted],
    index=X_selected_encoded.index
)

X_selected_sorted_df

**Mutual-information feature selection**

In [ ]:
from sklearn.feature_selection import mutual_info_regression, SelectKBest

num_features = 10

mi_selector = SelectKBest(score_func=mutual_info_regression,
                          k=num_features)
mi_selector.fit(X_selected_scaled, y)

mi_support = mi_selector.get_support()

mi_selected_features = X.columns[mi_support]
mi_scores = mi_selector.scores_[mi_support]

mi_feature_importance = sorted(zip(mi_selected_features, mi_scores),
                               key=lambda x: x[1], reverse=True)

mi_final_features, importances = zip(*mi_feature_importance)

plt.figure(figsize=(10, 6))
plt.barh(mi_final_features, importances, color='#336699')
plt.xlabel('Mutual Information Score')
plt.title('Top {} Features Chosen by Mutual Information'.format(num_features))
plt.gca().invert_yaxis()
plt.show()

Creating X_selected_sorted_df

In [ ]:
encoded_cols = X_selected_encoded.columns
name_to_idx = {col: i for i, col in enumerate(encoded_cols)}

idx_final = np.array([name_to_idx[f] for f in mi_final_features], dtype=int)

rfe_ranks_kept = rfe.ranking_[rfe.support_]
order = np.argsort(rfe_ranks_kept)
idx_final_sorted = idx_final[order]

X_selected_sorted = X_selected_scaled[:, idx_final_sorted]

print("Column indices of mi final features:", idx_final_sorted)
print("Ordered feature names:", encoded_cols[idx_final_sorted].tolist())

X_selected_sorted_df = pd.DataFrame(
    X_selected_sorted,
    columns=encoded_cols[idx_final_sorted],
    index=X_selected_encoded.index
)

X_selected_sorted_df

**boruta feature selection**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor

def boruta_feature_selection_kbest(
        X: pd.DataFrame,
        y,
        *,
        n_iters: int = 30,
        top_k: int = 10,
        rf_kwargs: dict | None = None,
        random_state: int | None = None
):
    rng = np.random.default_rng(random_state)
    rf_opts = dict(n_estimators=100, n_jobs=-1, random_state=random_state)
    if rf_kwargs:
        rf_opts.update(rf_kwargs)

    hits = np.zeros(X.shape[1], dtype=int)
    cols = X.columns.to_numpy()

    for _ in tqdm(range(n_iters), desc="Boruta iterations", unit="iter"):
        shadow = pd.DataFrame(
            rng.permutation(X.values),
            columns=[f"shadow_{c}" for c in cols],
            index=X.index
        )
        X_expanded = pd.concat([X, shadow], axis=1)

        rf = RandomForestRegressor(**rf_opts)
        rf.fit(X_expanded, y)

        importances = rf.feature_importances_
        max_shadow_imp = importances[len(cols):].max()
        hits += (importances[:len(cols)] > max_shadow_imp)

    idx_sorted = np.argsort(-hits)[:top_k]
    chosen_features = cols[idx_sorted]
    chosen_hits = hits[idx_sorted]

    order = np.argsort(chosen_hits)
    plt.figure(figsize=(8, 5))
    plt.barh(chosen_features[order], chosen_hits[order], color="#228B22")
    plt.gca().invert_yaxis()
    plt.xlabel("Hit count")
    plt.title(f"Boruta top {top_k} features")
    plt.show()

    return list(chosen_features)


Creating X_selected_sorted_df

In [ ]:
X_df = pd.DataFrame(X_selected_scaled, columns=X_selected_encoded.columns)
top10 = boruta_feature_selection_kbest(X_df, y, n_iters=30, top_k=10, random_state=42)
print(top10)
X_selected_sorted_df = X_df

**ElasticNet Feature Selection**

---



In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Parameters
k_final = 16
random_state = 42

# X_selected_scaled: your scaled data (numpy array)
# X_selected_encoded: your encoded DataFrame with column names
X0 = pd.DataFrame(X_selected_scaled, columns=X_selected_encoded.columns)
y0 = y

# ElasticNetCV pipeline
enet = make_pipeline(
    StandardScaler(with_mean=False),
    ElasticNetCV(l1_ratio=0.9, cv=5, random_state=random_state, max_iter=5000)
)
enet.fit(X0, y0)

# Extract and rank coefficients
abs_coefs = np.abs(enet[-1].coef_)
idx_sorted = np.argsort(-abs_coefs)[:min(k_final, len(abs_coefs))]
feat_elasticnet = X0.columns[idx_sorted]
X_enet_selected = X0.iloc[:, idx_sorted]

# Output
print(f"\nTop {k_final} features selected by ElasticNet:")
for i, f in enumerate(feat_elasticnet, 1):
    print(f"{i}. {f}")

# Optional: Return DataFrame of selected features
X_enet_selected_df = pd.DataFrame(X_enet_selected, columns=feat_elasticnet, index=X_selected_encoded.index)


Creating X_selected_sorted_df

In [ ]:
# Already computed earlier:
# - feat_elasticnet: top k_final feature names sorted by |ElasticNet coefficient|
# - X0: DataFrame with feature names
# - X_selected_scaled: scaled feature matrix
# - X_selected_encoded: original encoded DataFrame (with same column order)

# Map feature names to column indices
encoded_cols = X_selected_encoded.columns
name_to_idx = {col: i for i, col in enumerate(encoded_cols)}

# Get indices of selected features (already in order of importance)
idx_final_sorted = np.array([name_to_idx[f] for f in feat_elasticnet], dtype=int)

# Select sorted feature columns from the scaled matrix
X_selected_sorted = X_selected_scaled[:, idx_final_sorted]

# Print indices and names of selected features
print("Column indices of ElasticNet-selected features:", idx_final_sorted)
print("Ordered feature names:", encoded_cols[idx_final_sorted].tolist())

# Create DataFrame of the selected features
X_selected_sorted_df = pd.DataFrame(
    X_selected_sorted,
    columns=encoded_cols[idx_final_sorted],   # ordered feature names
    index=X_selected_encoded.index            # preserve the original row index
)

X_selected_sorted_df


**LASSO Feature Selection**

In [ ]:

# LASSO feature selection
lasso = Lasso(alpha=0.01, max_iter=100000)
lasso.fit(X_selected_scaled, y)

# Get the feature importance from the LASSO model
lasso_coef = lasso.coef_
lasso_importance = np.abs(lasso_coef)
lasso_importance_sorted = np.argsort(lasso_importance)[::-1]

# Select the top n features
selected_features = X.columns[lasso_importance_sorted[:num_features]]

for f, feature in enumerate(selected_features):
    print("%d. Feature %s" % (f + 1, feature))

In [ ]:
from sklearn.linear_model import Lasso
import numpy as np
import pandas as pd

def lasso_feature_selection_kbest(
    X: pd.DataFrame,
    y,
    *,
    alpha: float = 0.1,
    top_k: int = 10,
    max_iter: int = 100000,
    random_state: int = 42
):
    # Fit LASSO model
    lasso = Lasso(alpha=alpha, max_iter=max_iter, random_state=random_state)
    lasso.fit(X, y)

    # Coefficients and importance
    lasso_coef = lasso.coef_
    lasso_importance = np.abs(lasso_coef)

    # Sort feature indices by importance (descending)
    sorted_idx = np.argsort(lasso_importance)[::-1]

    # Select top_k non-zero features
    top_features_idx = [i for i in sorted_idx if lasso_coef[i] != 0][:top_k]
    selected_features = X.columns[top_features_idx]

    # Create sorted DataFrame
    X_selected_sorted_df = X[selected_features]

    # Optional: print top features
    for f, feature in enumerate(selected_features):
        print(f"{f + 1}. Feature: {feature}, Coef: {lasso_coef[X.columns.get_loc(feature)]:.4f}")

    return X_selected_sorted_df

# Example usage:
X_df = pd.DataFrame(X_selected_scaled, columns=X_selected_encoded.columns)
X_selected_sorted_df = lasso_feature_selection_kbest(X_df, y, top_k=10)


# Models

In [ ]:
X_selected_corrected = p_value_correction(X_selected_sorted_df, y)
X_selected_corrected.shape

In [ ]:
X_selected_encoded = one_hot_encoding(X_selected_sorted_df, 'LOC')

In [ ]:
X_selected_scaled = StandardScaler().fit_transform(X_selected_encoded)

In [ ]:

from sklearn.base import BaseEstimator, RegressorMixin
class KerasRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn, epochs=100, batch_size=32, verbose=0):
        self.build_fn = build_fn
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model_ = None

In [ ]:
loo = LeaveOneOut()

for train_idx, test_idx in loo.split(X_selected_scaled, y):
    print(f"Train set size: {len(train_idx)}")
    print(f"Test set size: {len(test_idx)}")
    break

In [ ]:
best_r2 = -np.inf
best_model = None
x=106

models = [
    #LinearRegression(),
    #Ridge(),
    #Lasso(),
    #ElasticNet(),
    #RandomForestRegressor(random_state=x),
    GradientBoostingRegressor(random_state=x),
    XGBRegressor(random_state=x),
    ExtraTreesRegressor(random_state=x),
    #SVR(),
    #DecisionTreeRegressor(random_state=x),
    #BayesianRidge(),
    #KNeighborsRegressor(),
    AdaBoostRegressor(random_state=x),
    #HuberRegressor(),
    #SGDRegressor(random_state=x),
    #KerasRegressor(build_modell, epochs=100, batch_size=32, verbose=0)
]

for model in models:
    print(f"Evaluating {type(model).__name__}:")

    r2_test, r2_train, mse_test, y_trains, y_train_preds, y_tests, y_test_preds = evaluate_model(X_selected_scaled, y, model, loo)

    if (r2_test > best_r2):
        best_r2 = r2_test
        best_model = model
print("Best model: ",  type(best_model).__name__)
print("Best R2: ",  best_r2)

In [ ]:

model = XGBRegressor(random_state=106)

In [ ]:
r2_test, r2_train, mse_test, y_trains, y_train_preds, y_tests, y_pred_tests = evaluate_model(X_selected_scaled, y, model, loo)

In [ ]:
def plot_prediction_graph2(y_test, y_pred_test, r2_test):

    _, p_value = pearsonr(y_test, y_pred_test)

    q_value = false_discovery_control(p_value, method='bh')

    plt.figure(figsize=(10, 6))

#4B878BFF, #D01C1FFF, #FF7057, #00B9AB

    sns.regplot(x=y_test, y=y_pred_test,
                scatter_kws=dict(color='#b300b3', alpha=0.6, s=33),
                line_kws=dict(color='#1E90FF', linewidth=2))
    plt.plot(np.linspace(min(y), max(y), 100), np.linspace(min(y), max(y), 100), color='black', linestyle='--', linewidth=1.3, label='Identity Line')
    plt.xlabel('Measured Dose (Gy/ GBq)', fontsize=18)
    plt.ylabel('Predicted Dose (Gy/ GBq)', fontsize=18)
    plt.text(0.48, 1.1, f'R² Test: {r2_test:.2f}', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes, fontsize=18, fontweight='bold')
    plt.text(0.48, 1.03, f'{"q-value: " + str(q_value) if q_value >= 0.05 else "(q-value < 0.05)"}', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes, fontsize=16, color='black')
    plt.grid(True, linewidth=0.5)
    plt.legend(loc='upper left', fontsize=18)
    #plt.suptitle('Validation Cohort', fontsize=18, fontweight='bold')

    plt.tight_layout()
    plt.show()


In [ ]:
plot_prediction_graph2(y_tests, y_pred_tests, r2_test)

In [ ]:
#evaluate_based_on_feature_number(X_selected_scaled, y, RandomForestRegressor(random_state=0), kf)

In [ ]:
import re

def get_simple_feature_name(feature_names):
    simple_names = []
    for feature_name in feature_names:
        simple_name = re.sub(r'\(.*?\)', '', feature_name)
        simple_name = re.sub(r'\[.*?\]', '', simple_name)

        simple_name = simple_name.strip()

        if simple_name in simple_names:
            last_underscore_index = feature_name.rfind('_')
            simple_name = feature_name if last_underscore_index == -1 else feature_name[last_underscore_index:]
        simple_names.append(simple_name)
    return simple_names


In [ ]:
feature_names = X.columns
X_selected_scaled_df = pd.DataFrame(X_selected_scaled, columns=feature_names)
X_selected_scaled_df

X_selected_scaled_df.columns = get_simple_feature_name(X_selected_scaled_df.columns)
X_selected_scaled_df.shape


In [ ]:
import shap
import matplotlib.pyplot as plt

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_selected_scaled_df)

plt.figure()
shap.summary_plot(shap_values, X_selected_scaled_df, show=False)

plt.gca().tick_params(axis='y', labelsize=17)

plt.gca().tick_params(axis='x', labelsize=17)

cbar = plt.gcf().axes[-1]
cbar.tick_params(labelsize=17)
cbar.set_ylabel("Feature Value", fontsize=17)

ax = plt.gca()
ax.set_xlabel("SHAP Value (Impact on Model Output)", fontsize=17)

plt.show()


In [ ]:

import shap
import matplotlib.pyplot as plt
import numpy as np

mean_abs_shap_values = np.abs(shap_values).mean(axis=0)
most_important_feature_index = np.argmax(mean_abs_shap_values)
most_important_feature_name = X_selected_scaled_df.columns[most_important_feature_index]

shap_feature_values = shap_values[:, most_important_feature_index]
feature_values = X_selected_scaled_df.iloc[:, most_important_feature_index]

plt.figure(figsize=(10, 6))

sc = plt.scatter(feature_values, shap_feature_values, c=feature_values, cmap="bwr", edgecolors='k')

cbar = plt.colorbar(sc)
cbar.set_label("Feature Value", fontsize=15)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel(f"{most_important_feature_name} Value", fontsize=15)
plt.ylabel("SHAP Value (Impact on Model Output)", fontsize=15)
#plt.title(f"Relationship Between {most_important_feature_name} and SHAP Values", fontsize=15)

plt.show()


In [ ]:
import re

def get_simple_feature_name2(feature_names):
    simple_names = []
    for feature_name in feature_names:
        simple_name = re.sub(r'^(INTENSITY-HISTOGRAM_)', 'IH-', feature_name)
        simple_name = re.sub(r'^(INTENSITY-BASED_)', 'IB-', simple_name)
        simple_name = re.sub(r'\(.*?\)', '', simple_name)
        simple_name = re.sub(r'\[.*?\]', '', simple_name)

        simple_name = simple_name.strip()
        if simple_name in simple_names:
            last_underscore_index = feature_name.rfind('_')
            simple_name = feature_name if last_underscore_index == -1 else feature_name[last_underscore_index:]
        simple_names.append(simple_name)
    return simple_names


In [ ]:
feature_names = X.columns
X_selected_scaled_df = pd.DataFrame(X_selected_scaled, columns=feature_names)
X_selected_scaled_df

X_selected_scaled_df.columns = get_simple_feature_name2(X_selected_scaled_df.columns)
X_selected_scaled_df.shape

In [ ]:
pip install lime

In [ ]:
import numpy as np
import shap
import lime.lime_tabular
import matplotlib.pyplot as plt
import textwrap
from math import pi

# model = XGBRegressor()
# model.fit(X_train, y_train)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_selected_scaled_df)

shap_importances = np.abs(shap_values).mean(axis=0)

lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    X_selected_scaled_df.values,
    feature_names=X_selected_scaled_df.columns.tolist(),
    verbose=True,
    mode="regression",
    random_state=106

)

lime_exp = lime_explainer.explain_instance(X_selected_scaled_df.iloc[0].values, model.predict, num_features=8)

lime_weights_dict = dict(lime_exp.as_map()[0])

lime_importances = np.array([lime_weights_dict.get(i, 0) for i in range(len(X_selected_scaled_df.columns))])

shap_importances /= shap_importances.max()
lime_importances = np.abs(lime_importances) / np.abs(lime_importances).max()

feature_names = X_selected_scaled_df.columns

shap_importances = np.append(shap_importances, shap_importances[0])
lime_importances = np.append(lime_importances, lime_importances[0])

feature_names_wrapped = ["\n".join(textwrap.wrap(name, width=15)) for name in feature_names]

num_vars = len(feature_names)
angles = np.linspace(0, 2 * pi, num_vars, endpoint=False).tolist()
angles.append(angles[0])

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

ax.fill(angles, shap_importances, color='#C71585', alpha=0.25, label="SHAP")
ax.plot(angles, shap_importances, color='#663399', linewidth=2)

ax.fill(angles, lime_importances, color='#87CEEB', alpha=0.25, label="LIME")
ax.plot(angles, lime_importances, color='#1E90FF', linewidth=2)

ax.set_xticks(angles[:-1])
for label, angle in zip(feature_names_wrapped, angles[:-1]):
    rotation = np.degrees(angle)

    if 240 < rotation < 280:
        ha = "center"
        rotation += 90
    elif 70 < rotation < 110:
        ha = "center"
        rotation += 270
    elif 110 < rotation < 270:
        ha = "right"
        rotation += 180
    else:
        ha = "left"

    ax.text(angle, 1.2, label, rotation=rotation, size=17, ha=ha, va="center")

ax.set_rlabel_position(0)
plt.yticks([0.2, 0.6, 1.0], ["0.2", "0.6", "1.0"], color="grey", size=10)
plt.ylim(0, 1.1)
ax.grid(color='#003399', linestyle=':', linewidth=1)

plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1), fontsize=17)

plt.show()


In [ ]:
import numpy as np
import shap
import lime.lime_tabular
import matplotlib.pyplot as plt
import textwrap
from math import pi

# model = XGBRegressor()
# model.fit(X_train, y_train)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_selected_scaled_df)

shap_importances = np.abs(shap_values).mean(axis=0)

lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    X_selected_scaled_df.values,
    feature_names=X_selected_scaled_df.columns.tolist(),
    verbose=True,
    mode="regression",
    random_state=106

)

lime_exp = lime_explainer.explain_instance(X_selected_scaled_df.iloc[0].values, model.predict, num_features=10)

lime_weights_dict = dict(lime_exp.as_map()[0])

lime_importances = np.array([lime_weights_dict.get(i, 0) for i in range(len(X_selected_scaled_df.columns))])

shap_importances /= shap_importances.max()
lime_importances = np.abs(lime_importances) / np.abs(lime_importances).max()

feature_names = X_selected_scaled_df.columns

shap_importances = np.append(shap_importances, shap_importances[0])
lime_importances = np.append(lime_importances, lime_importances[0])

feature_names_wrapped = ["\n".join(textwrap.wrap(name, width=9)) for name in feature_names]

num_vars = len(feature_names)
angles = np.linspace(0, 2 * pi, num_vars, endpoint=False).tolist()
angles.append(angles[0])

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

ax.fill(angles, shap_importances, color='#C71585', alpha=0.25, label="SHAP")
ax.plot(angles, shap_importances, color='#663399', linewidth=2)

ax.fill(angles, lime_importances, color='#87CEEB', alpha=0.25, label="LIME")
ax.plot(angles, lime_importances, color='#1E90FF', linewidth=2)

ax.set_xticks(angles[:-1])
for label, angle in zip(feature_names_wrapped, angles[:-1]):
    rotation = np.degrees(angle)

    if 240 < rotation < 280:
        ha = "center"
        rotation += 90
    elif 90 < rotation < 110:
        ha = "center"
        rotation += 270
    elif 110 < rotation < 270:
        ha = "right"
        rotation += 180
    else:
        ha = "left"

    ax.text(angle, 1.2, label, rotation=rotation, size=17, ha=ha, va="center")

ax.set_rlabel_position(0)
plt.yticks([0.2, 0.6, 1.0], ["0.2", "0.6", "1.0"], color="grey", size=10)
plt.ylim(0, 1.1)
ax.grid(color='#003399', linestyle=':', linewidth=1)

plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1), fontsize=17)

plt.show()


In [ ]:
import numpy as np
import shap
import lime.lime_tabular
import matplotlib.pyplot as plt
import textwrap
from math import pi

# model = XGBRegressor()
# model.fit(X_train, y_train)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_selected_scaled_df)

shap_importances = np.abs(shap_values).mean(axis=0)

lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    X_selected_scaled_df.values,
    feature_names=X_selected_scaled_df.columns.tolist(),
    verbose=True,
    mode="regression",
    random_state=106

)

lime_exp = lime_explainer.explain_instance(X_selected_scaled_df.iloc[0].values, model.predict, num_features=16)

lime_weights_dict = dict(lime_exp.as_map()[0])

lime_importances = np.array([lime_weights_dict.get(i, 0) for i in range(len(X_selected_scaled_df.columns))])

shap_importances /= shap_importances.max()
lime_importances = np.abs(lime_importances) / np.abs(lime_importances).max()

feature_names = X_selected_scaled_df.columns

shap_importances = np.append(shap_importances, shap_importances[0])
lime_importances = np.append(lime_importances, lime_importances[0])

feature_names_wrapped = ["\n".join(textwrap.wrap(name, width=9)) for name in feature_names]

num_vars = len(feature_names)
angles = np.linspace(0, 2 * pi, num_vars, endpoint=False).tolist()
angles.append(angles[0])

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

ax.fill(angles, shap_importances, color='#C71585', alpha=0.25, label="SHAP")
ax.plot(angles, shap_importances, color='#663399', linewidth=2)

ax.fill(angles, lime_importances, color='#87CEEB', alpha=0.25, label="LIME")
ax.plot(angles, lime_importances, color='#1E90FF', linewidth=2)

ax.set_xticks(angles[:-1])
for label, angle in zip(feature_names_wrapped, angles[:-1]):
    rotation = np.degrees(angle)

    if 250 < rotation < 280:
        ha = "right"
        rotation += 90
    elif 80 < rotation < 110:
        ha = "center"
        rotation += 270
    elif 110 < rotation < 270:
        ha = "right"
        rotation += 180
    else:
        ha = "left"

    ax.text(angle, 1.22, label, rotation=rotation, size=17, ha=ha, va="center")

ax.set_rlabel_position(0)
plt.yticks([0.2, 0.6, 1.0], ["0.2", "0.6", "1.0"], color="grey", size=10)
plt.ylim(0, 1.1)
ax.grid(color='#003399', linestyle=':', linewidth=1)

plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1), fontsize=17)

plt.show()
